In [ ]:
# default_exp main_notebook

# Main

>  Add some stuff here ...


In [4]:
#hide
from nbdev.showdoc import *
def generate_python():
    from nbdev.export import notebook2script; notebook2script()
    
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# export
# Setup in notebook flag.  Useful when you export code and have some situations where you dont want certain part of code run like visulaizations..
import sys,os
try: from nbdev.imports import IN_NOTEBOOK
except: IN_NOTEBOOK=False

if IN_NOTEBOOK :
    print("In Notebooke mode")
else :
    print("Running in batch mode")
    os.chdir("../my_python_code")

In Notebooke mode


In [15]:
#export
import my_python_code.library_notebook as funcs


In [16]:
funcs.hello_world("Test")

you just ran a function in library_notebook.py.  You passed Test


In [17]:
# export
print(f"IN_NOTEBOOK = {IN_NOTEBOOK}")

IN_NOTEBOOK = True


In [18]:
generate_python()

Converted 00_funcs.ipynb.
Converted 01_main.ipynb.


In [19]:
! git commit -am "update from main.ipynb"

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
	.gitattributes
	.gitconfig
	.gitignore.orig

nothing added to commit but untracked files present


In [20]:
! git push origin main

Everything up-to-date


In [21]:
! python my_python_code/main_notebook.py

Running in batch mode
Traceback (most recent call last):
  File "my_python_code/main_notebook.py", line 17, in <module>
    import my_python_code.library_notebook as funcs
ModuleNotFoundError: No module named 'my_python_code'
